# The N-body problem

### Imports

In [ ]:
# Magic function: enables interactive plot
%matplotlib widget
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


### Initial conditions

In [ ]:
# body mA initial conditions
mA = 1e26  # mass (kg)
rA0 = np.array([1E3, 0, 0])  # initial position (km)
vA0 = np.array([10, -20, 10])  # initial velocity (km/s)

# body mB initial conditions
mB = 1e26  # mass (kg)
rB0 = np.array([-1E3, 0, 0])  # initial position (km)
vB0 = np.array([10, 40, 10])  # initial velocity (km/s)

# body mC initial conditions
mC = 1e26  # mass (kg)
rC0 = np.array([0, 0, 0])  # initial position (km)
vC0 = np.array([0, 20, 0])  # initial velocity (km/s)

In [ ]:
y0 = np.stack([
    np.stack([rA0, vA0]),
    np.stack([rB0, vB0]),
    np.stack([rC0, vC0])
])

In [ ]:
masses = [mA, mB, mC]

In [ ]:
y0.shape

In [ ]:
G = 6.67259e-20  # Gravitational constant (km**3/kg/s**2)

### The derivatives


In [ ]:
def n_body_eqm_derivatives(_y, t, _G, masses):
    """
    derivatives of the equations of motion describing the two-body system
    t is unused, but we keep it for consistency with scipy requirement
    """
    derivatives = []
    for i in range(_y.shape[0]):
        ri = _y[i, 0, :]
        mi = masses[i]
        vi = _y[i, 1, :]

        # acceleration
        ai = _G * mi * sum([
            (_y[j, 0, :] - ri) / np.linalg.norm(_y[j, 0, :] - ri)**3
        for j in {range(_y.shape[0])} - {i}
        ])
        derivatives.append(np.stack([vi, ai]))

    derivatives = np.stack(derivatives)

    return derivatives


### Forward time evolution

In [ ]:
dt = 0.001  # time step (s)
tf = 1E2  # end of simulation (s)

In [ ]:
def evolve(y0, tf, dt, method, params):
    history = []
    yn = y0
    t_axis = np.arange(0, tf, dt)
    for tn in t_axis:
        yn = evolve_one_step(yn, tn, dt, method, params)
        history.append(yn.copy())

    history = np.stack(history, axis=-1)
    return history

### Euler's method

In [ ]:
def evolve_one_step(yn, tn, dt, method, params):
    if method == "euler":
        f = n_body_eqm_derivatives(yn, tn, *params)
        yn += f * dt
    elif method == "rk4":
        f1 = n_body_eqm_derivatives(yn, tn, *params)
        f2 = n_body_eqm_derivatives(yn + f1 * dt / 2, tn + dt / 2, *params)
        f3 = n_body_eqm_derivatives(yn + f2 * dt / 2, tn + dt / 2, *params)
        f4 = n_body_eqm_derivatives(yn + f3 * dt, tn + dt, *params)
        yn += (f1 + 2 * f2 + 2 * f3 + f4) * dt / 6

    return yn

### Running the simulation

In [ ]:
history = evolve(y0, tf, dt, "euler", params=(G, masses))

In [ ]:
# if we want to compare with odeint from scipy
#from scipy.integrate import odeint
#history = odeint(n_body_eqm_derivatives, y0, np.arange(0, tf, dt), args=(G, masses))

In [ ]:
history.shape

### Plotting the outcomes

In [ ]:
# Trajectories
trajectories = history[:, 0, :, :]
trajectories.shape

In [ ]:
plt.style.use('dark_background')

fig = plt.figure()
ax = plt.axes(projection='3d')
colors = mpl.colormaps["Set3"]

def animate(frame_num):
    ax.clear()
    for i in range(trajectories.shape[0]):
        trajectory_i = trajectories[i, :, :]
        traj_x_i = trajectory_i[0, :]
        traj_y_i = trajectory_i[1, :]
        traj_z_i = trajectory_i[2, :]

        ax.plot3D(traj_x_i[:frame_num], traj_y_i[:frame_num], traj_z_i[:frame_num], c=colors[i])
        ax.scatter(traj_x_i[frame_num], traj_y_i[frame_num], traj_z_i[frame_num], c=colors[i], marker='o')

    for j, dim_label in enumerate(["X", "Y", "Z"]):
        dim_values = trajectories[:, j, :]
        min_value = np.min(dim_values)
        max_value = np.max(dim_values)
        ax.set(xlim3d=(min_value, max_value), xlabel=dim_label)



anim = FuncAnimation(fig, animate, frames=history.shape[-1], interval=10, repeat=False)
plt.show()